In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

from quick_pp.objects import Project

# Load well from saved file
project_name = "VOLVE"
project_path = rf"data\04_project\{project_name}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
# Load data
well_name = '15-9-19-BT2'
well = project.get_well(well_name)
well_data = well.data.copy()

# Water Saturation Estimation

Water saturation estimation is crucial in petrophysics for several reasons:

1. **Hydrocarbon Volume Calculation**: It helps determine the volume of hydrocarbons in place. Accurate water saturation (Sw) values are essential for calculating the original oil in place (OOIP) and original gas in place (OGIP) volumes¹(https://petroshine.com/fluid-saturation/).
2. **Reservoir Characterization**: Understanding the distribution of water saturation helps in characterizing the reservoir, which is vital for planning production strategies and enhancing recovery¹(https://petroshine.com/fluid-saturation/).
3. **Production Forecasting**: Sw values are used in reservoir models to predict future production and to evaluate the economic viability of the reservoir²(https://www.mdpi.com/2077-1312/9/6/666).

### Methods to Estimate Water Saturation

1. **Resistivity Logs**: This is the most common method, where water saturation is estimated using resistivity measurements from well logs. The Archie equation is often used for clean sands, while modified versions like the Waxman-Smits model are used for shaly sands³(https://petrowiki.spe.org/Water_saturation_determination).
2. **Capillary Pressure Measurements**: Laboratory measurements of capillary pressure and corresponding water saturation provide detailed information about the pore structure and fluid distribution³(https://petrowiki.spe.org/Water_saturation_determination).
3. **Core Analysis**: Direct measurement of water saturation from core samples using techniques like the Dean-Stark method³(https://petrowiki.spe.org/Water_saturation_determination).
4. **Nuclear Magnetic Resonance (NMR)**: NMR logging tools can provide estimates of water saturation by measuring the response of hydrogen nuclei in the formation fluids³(https://petrowiki.spe.org/Water_saturation_determination).

This notebook estimates the water saturation using Archie equation and saturation height function based on the capillary pressure measurement.


***
# Log Derived Water Saturation

Estimation of Rw based on formation water salinity, assuming the depths are already in True Vertical Depth Sub Sea (TVDSS).
The range of Rw used in the original paper is 0.015 to 0.03 ohm.m

Estimation of cementation factor (m) based on pickett plot.

In [ ]:
from ipywidgets import widgets, interact

from quick_pp.saturation import pickett_plot

# water_wells = ['HW-5', 'HW-7', 'HW-8', 'HW-9', 'HW-28', 'HW-31']
# focused_data = all_data[all_data.WELL_NAME.isin(water_wells)].copy()
focused_data = well_data.copy()

wells = widgets.SelectMultiple(
    options=['All'] + list(focused_data['WELL_NAME'].unique()),
    value=['All'],
    description='Wells:'
)
m = widgets.FloatSlider(
    value=2,
    min=1,
    max=3,
    step=.1,
    readout_format='.1f'
)
min_rw = widgets.FloatSlider(
    value=.01,
    min=.001,
    max=.1,
    step=.001,
    readout_format='.3f'
)
min_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.min(),
    min=focused_data.DEPTH.min(),
    max=focused_data.DEPTH.max() - 10,
    step=.1,
    readout_format='.1f'
)
max_depth = widgets.FloatSlider(
    value=focused_data.DEPTH.max(),
    min=focused_data.DEPTH.min() + 10,
    max=focused_data.DEPTH.max(),
    step=.1,
    readout_format='.1f'
)

@interact(wells=wells, m=m, min_rw=min_rw, min_depth=min_depth, max_depth=max_depth)
def param(wells, m, min_rw, min_depth, max_depth):
    if 'All' in wells:
        data = focused_data[(focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    else:
        data = focused_data[(focused_data.WELL_NAME.isin(wells)) & (focused_data.DEPTH >= min_depth) & (focused_data.DEPTH <= max_depth)]
    pickett_plot(data['RT'], data['PHIT'], m=m, min_rw=min_rw, title=f'Pickett Plot for {wells[0]}')

In [ ]:
import numpy as np
from quick_pp.saturation import *

water_salinity = 1e4
m = 1.9

temp_grad = estimate_temperature_gradient(well_data['TVD'], 'metric')
rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)

swt = archie_saturation(well_data['RT'], rw, well_data['PHIT'], m=m, n=2)
# swt = swt.clip(0, 1.1)

fig, axes = plt.subplots(2, 1, figsize=(15, 3), sharex=True)
axes[0].plot(well_data['DEPTH'], swt, label='SWT')
axes[0].plot(well_data['DEPTH'], well_data['SW'], label='SW')
axes[0].plot(well_data['DEPTH'], np.ones(len(well_data)), color='black', linestyle='--')
axes[0].set_ylim(0, 2)
axes[0].legend()

axes[1].plot(well_data['DEPTH'], rw, label='RW')
axes[1].set_yscale('log')
axes[1].legend()
fig.tight_layout()

***
# Plot the results

In [ ]:
from quick_pp.plotter.plotter import plotly_log

# Plot individual results
well_data['SWT'] = swt
fig = plotly_log(well_data, well_name=well_name, depth_uom='m')
fig.show(config=dict(scrollZoom=True))

# Apply to all

In [ ]:
water_salinity = 1e4
m = 2

df = project.get_all_data()
for well_name, plot_data in df.groupby('WELL_NAME'):

    temp_grad = estimate_temperature_gradient(plot_data['TVD'], 'metric')
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    swt = archie_saturation(plot_data['RT'], rw, plot_data['PHIT'], m=m, n=2)

    plot_data['SWT'] = swt.clip(0, 1)
    
    # Save the well data
    project.update_data(plot_data)
    project.save()